## Import libraries

In [7]:
# solve error #15
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import logging
import random
from datetime import datetime
import os
import torch
import numpy as np
import params as p

from data_augmentation import DataAugmentor_v1
from data_generator import DataGeneratorPommerman
from dqn import DQN
from models import Pommer_Q
from util.data import transform_observation_centralized

## Define helper functions

In [8]:
board_size = 11
obs_size =  board_size*2-1
transform_func =  transform_observation_centralized

q = Pommer_Q(obs_size, transform_func)
q_target = Pommer_Q(obs_size, transform_func)
algo = DQN(q, q_target)

def train_model(algo, num_iter):
    torch.manual_seed(p.seed)
    np.random.seed(p.seed)
    random.seed(p.seed)

    data_generator = DataGeneratorPommerman(
	p.env,
	augmenter=[
            # DataAugmentor_v1()
        ])

    for i in range(num_iter):
        print(f"Iteration {i+1}/{num_iter} started")
        # generate data
        policy = algo.get_policy()
        data_generator.generate(p.episodes_per_iter, policy, q.get_transformer())

        # train model on data
        for j in range(p.gradient_steps_per_iter):
            if p.episode_backward:
                batch = data_generator.get_episode_buffer()
            else:
                batch = data_generator.get_batch_buffer(p.batch_size)
            algo.train(batch)
 
        print("------------------------")
        if i % p.intermediate_test == p.intermediate_test-1:
            print("Testing model")
            algo.set_train(False)
            policy = algo.get_policy()
            data_generator.generate(p.episodes_per_eval, policy, q.get_transformer(), render=p.render_tests)
            algo.set_train(True)
            print("------------------------")
    return algo



In [10]:
train_model(algo, 10)

Iteration 1/10 started


FileNotFoundError: [Errno 2] No such file or directory: 'data/log'